In [ ]:
#pip install nltk
#conda install nltk
import nltk
#nltk.download()

In [ ]:
# nltk.download()

In [ ]:
from nltk.corpus import brown

In [ ]:
brown.words()

# Tokenizing

In [ ]:
text = 'Python is a programing language. We are learning python.'

In [ ]:
s = text.split('.')

In [ ]:
s

In [ ]:
s[0].split()

In [ ]:
text = 'Mr. Python is a programing language. We are learning python.'

In [ ]:
s = text.split('.')

In [ ]:
s

In [ ]:
from nltk import word_tokenize,sent_tokenize

In [ ]:
sent_tokenize(text)

In [ ]:
word_tokenize(text)

# Text normalization

In [ ]:
import string

In [ ]:
puncts = string.punctuation

In [ ]:
puncts

In [ ]:
text_tokens = word_tokenize(text)

In [ ]:
text_tokens_f = [w for w in text_tokens if w not in puncts]

In [ ]:
text_tokens_f

In [ ]:
from nltk.corpus import stopwords

In [ ]:
s_words = stopwords.words('english')

In [ ]:
len(s_words)

In [ ]:
s_words

In [ ]:
text_tokens_f = [w for w in text_tokens_f if w.lower() not in s_words]
# [w for w in text_tokens_f if w.lower() not in s_words]

In [ ]:
text_tokens_f

In [ ]:
from nltk.stem import PorterStemmer #worked -> work

In [ ]:
ps = PorterStemmer()

In [ ]:
ps.stem('dogs')

In [ ]:
ps.stem('leaves')

In [ ]:
from nltk.stem.wordnet import WordNetLemmatizer

In [ ]:
wnl = WordNetLemmatizer()

In [ ]:
wnl.lemmatize('leaves')

# Tasks

In [ ]:
nltk.corpus.gutenberg.fileids()

In [ ]:
mma = nltk.corpus.gutenberg.words('austen-emma.txt')

In [ ]:
mma[:25]

1) Lemmatize all the tokens in the text

In [ ]:
def print_preview(a: list):
    print(len(a))
    print(a[:25])

In [ ]:
lemmas = [wnl.lemmatize(w) for w in mma]
print_preview(lemmas)

2) Remove all stop words from the text.

In [ ]:
no_stop_words = [w for w in lemmas if w not in s_words]
print_preview(no_stop_words)

3) Find all the unique tokens in the text.

In [ ]:
uniques = list(set(no_stop_words))
print_preview(uniques)

4) Create a mapping word2index which maps every unique token to an integer index. Create a reverse mapping index2word.  

In [ ]:
sorted_uniques = sorted(uniques)
print_preview(sorted_uniques)
word2index = {sorted_uniques[index]: index for index in range(len(sorted_uniques))}
index2word = {index: word for word, index in word2index.items()}
print(index2word.get(0))
print(word2index.get("Am"))

5) Considering a window size of 3, create a token-token frequency matrix where each cell notes the frequency of occurrence. 

In [ ]:
frequency_matrix = [[0 for j in range(len(uniques))] for i in range(len(uniques))]
print(len(frequency_matrix))
print(len(frequency_matrix[0]))
for i, word in enumerate(mma):
    if word not in uniques:
        continue
    context = mma[max(i-3,0):i] + mma[i+1:min(i+4,len(mma))]
    for context_word in context:
        if context_word in uniques:
            frequency_matrix[word2index.get(word)][word2index.get(context_word)] += 1

In [ ]:
print(frequency_matrix[:10][:10])

6) Compute Positive pointwise mutual information for each token pair in the matrix.

In [ ]:
import math

In [ ]:
test_matrix = [
    [0, 0, 1, 0, 1],
    [0, 0, 1, 0, 2],
    [2, 1, 0, 1, 0],
    [1, 6, 0, 4, 0],
]

In [ ]:
frequency_matrix = frequency_matrix

sum_of_all_entries = sum([sum(a) for a in frequency_matrix])
sum_rows = [sum(frequency_matrix[i]) for i in range(len(frequency_matrix))]
sum_columns = [sum([frequency_matrix[i][j] for i in range(len(frequency_matrix))]) for j in range(len(frequency_matrix[0]))]

print(sum_of_all_entries)
print(sum_rows[:10])
print(sum_columns[:10])

def get_P_wi(i: int):
    sum_row = sum_rows[i]
    return sum_row / sum_of_all_entries

def get_P_cj(j: int):
    sum_column = sum_columns[j]
    return sum_column / sum_of_all_entries

def get_P_wi_cj(i: int, j: int):
    return frequency_matrix[i][j] / sum_of_all_entries

def calculate_pmi(i: int, j: int):
    try:
        numerator = get_P_wi_cj(i, j)
        denominator = (get_P_wi(i) * get_P_cj(j))
        if denominator == 0 or numerator == 0:
            return 0
        return math.log2(numerator / denominator)
    except Exception as e:
        print(f"Exception: {e}")
        print(f"Denominator: {denominator}")

def calculate_ppmi(i: int, j: int):
    return max(calculate_pmi(i, j), 0)

In [ ]:
ppmi = calculate_ppmi(0, 0)
print(ppmi)

In [ ]:
# ppmi_matrix = [[0.0 for j in range(len(frequency_matrix[i]))] for i in range(len(frequency_matrix))]
ppmi_matrix = [[0.0 for j in range(1000)] for i in range(1000)]

for i in range(len(ppmi_matrix)):
    for j in range(len(ppmi_matrix[i])):
        print(f"Calculating ppmi for {i}, {j}")
        ppmi_matrix[i][j] = calculate_ppmi(i, j)

print(ppmi_matrix)